In [ ]:
!pip install accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [ ]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


access_token = ""


quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", quantization_config=quantization_config, token=access_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
def generate_questions_from_answers(text_narrations_list):
  #print(text_narrations_list)
  questions = []
  text=""
  # we need better model
  '''
  example
   1. C irons cloth on a table
   2. C interacts with X
   '''
  for i in range(len(text_narrations_list)):
    text+= f"{i+1}. " + text_narrations_list[i] + "\n"


  input_text = f"""
Given the following narrations describing the actions of a person, generate {len(text_narrations_list)} independent, specific queries (one per line per narration) in the past tense that could be answered by looking at the narrations.
Use 'wh' questions such as Who, Where, Which, What, Whom, Whose, When, and How many to produce smart questions. Ensure the questions are clear by focusing on the object and verb of each narration.
Replace any mention of 'C' or '#C' in the narration with 'I' in the questions.

**************************************
EXAMPLE:
Narrations:
1. #C C ironed cloth on a table
2. #C C interacted with X
3. #C C opened the top of the refrigerator with her right hand
4. #C C picked up the knife with her right hand
5. #C C put the book on the shelf
6. #C C saw a cat in the garden

Corresponding Questions:
1. What did I iron on the table?
2. Who did I interact with?
3. What hand did I use to open the top of the refrigerator?
4. What did I do with the knife?
5. Where did I put the book?
6. In what location did I see the cat?
**************************************

Now it is your turn
Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  generated_text=""
  num_of_tries = 0
  while generated_text.count("?") +1 < len(text_narrations_list):

    if num_of_tries >= 2:
      outputs = model.generate(**input_ids, max_length=1300)
      generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
      break

    outputs = model.generate(**input_ids, max_length=1300, repetition_penalty=1.1, temperature=0.8, top_k=50, top_p=0.9)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    num_of_tries +=1

  #print(generated_text)
  trimmed_text = generated_text[generated_text.rfind(":")+1:]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q.strip()[3:])

  #print(questions)
  #print(text_narrations_list)
  return questions

We will save the big boy and then run these codes on separate notebooks

In [5]:
# we are going to work with a copy
!cp Ours_mw6.json Ours_mw6_copy.json

In [14]:
import json

big_boy = json.load(open("/content/Ours_mw6_copy.json"))
big_boy_vids = big_boy["videos"]

slice_size = 96

# Initialize the list to hold the slices
slices_list = []

# Loop through the original list and create slices
for i in range(0, len(big_boy_vids), slice_size):
    slice_part = big_boy_vids[i:i + slice_size]
    slices_list.append(slice_part)

In [15]:
print("Insert an integer between 0 and", len(slices_list)-1)

Insert an integer between 0 and 23


In [ ]:
SPLIT_INDEX = "<change here>" # change here !!!
used_vids = slices_list[SPLIT_INDEX]

TypeError: list indices must be integers or slices, not str

In [ ]:
for vid in used_vids:
  for clip in vid["clips"]:
    for annotator in clip["annotations"]:
      language_queries = annotator["language_queries"]
      answers = [item["query"] for item in language_queries]
      questions = generate_questions_from_answers(answers)

      while len(questions) < len(language_queries):
        questions.append("") # empty but at least no bug!!!

      for i in range(len(language_queries)):
        language_queries[i]["query"] = questions[i]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['#C C unwraps a scarf on the mannequin with her right hand.', '#C C spreads the scarf on the mannequin with both hands.', '#C C hangs the scarf on the mannequin with her right hand.', '#C C walks towards a mannequin.', '#C C walks towards a shelf.', '#C The man A converses with C.', '#C C holds a phone pouch in her hands.', '#C C covers the phone pouch in her left hand with the phone pouch cover in her right hand', '#C The man A inserts the phone in his right hand into the pouch in the hands of C.', '#C C holds the phone pouch cover in her right hand.', '#C C touches her face with her left hand. #unsure', '#C C walks towards the table.', '#C C touches a muffler on the table with her left hand.', '#C C takes a muffler from the table with her right hand.', '#C C holds the muffler with both hands.']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 15
  simple queries (one per line per narration) that could be answered by looking at the narrations.
  Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 15 questions.

  ******************************************
  EXAMPLE:
  Narrations:
  1. #C C irons cloth on a table
  2. #C C interacts with X

  Corresponding Questions:
  1. What does C iron?
  2. Who does C interact with?
  ******************************************

  Now it is your turn
  Narrations:
  1. #C C unwraps a scarf on the mannequin with her right hand.
2. #C C spreads the scarf on the mannequin with both hands.
3. #C C hangs the scarf on the mannequin with her right hand.
4. #C C walks towards a mannequin.
5. #C C walks towards a shelf.
6. #C The man A converses with C.
7. #C C holds a phone pouch in her hands.
8. #C C covers the phone pouch in her left hand with the phon

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Given the following narrations describing the actions of a person, generate independent from one another 15
  simple queries (one per line per narration) that could be answered by looking at the narrations.
  Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 15 questions.

  ******************************************
  EXAMPLE:
  Narrations:
  1. #C C irons cloth on a table
  2. #C C interacts with X

  Corresponding Questions:
  1. What does C iron?
  2. Who does C interact with?
  ******************************************

  Now it is your turn
  Narrations:
  1. #C C touches #unsure
2. #C C converses with man X
3. #C C touches #unsure
4. #C C converses with man X
5. #C C touches wallet
6. #C C converses with man X
7. #C C picks the hat on top of the shelf
8. #C C picks the  cloth on top of the shelf
9. #O a man X removes a shirt from a wall with his right hand.
10. #C C hangs a piece of clothing on the cloth rail with both hands.
11. #C C 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 15
  simple queries (one per line per narration) that could be answered by looking at the narrations.
  Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 15 questions.

  ******************************************
  EXAMPLE:
  Narrations:
  1. #C C irons cloth on a table
  2. #C C interacts with X

  Corresponding Questions:
  1. What does C iron?
  2. Who does C interact with?
  ******************************************

  Now it is your turn
  Narrations:
  1. #C C holds the fur fabric
2. #C C holds the rope with both hands
3. #C C stands at the door
4. #C C holds the belt
5. #C C looks at the door
6. #C C opens the door
7. #C C looks at the dog
8. #C C enters into the bed room
9. #C C holds the dog
10. #C C looks on the bed
11. #C C walks in the house
12. #C C holds the fur fabric
13. #C C looks at the living room
14. #C C holds the fur

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given the following narrations describing the actions of a person, generate independent from one another 15
  simple queries (one per line per narration) that could be answered by looking at the narrations.
  Use where, what, when and who questions only. Don't ask for feedback, you MUST only generate 15 questions.

  ******************************************
  EXAMPLE:
  Narrations:
  1. #C C irons cloth on a table
  2. #C C interacts with X

  Corresponding Questions:
  1. What does C iron?
  2. Who does C interact with?
  ******************************************

  Now it is your turn
  Narrations:
  1. #C C looks at the dog
2. #C C holds the dog
3. #C C holds the bucket
4. #C C looks on the living room
5. #C C holds the water tap
6. #C C looks at the oven
7. #C C stands in the kitchen
8. #C C holds the phone
9. #C C walks on the collider
10. #C C looks through the mirror
11. #C C holds the phone
12. #C C looks at the dog
13. #C C operates the phone
14. #C C looks on the walls
15.

In [ ]:
#Save the part
new_file_name ="ours_mw6_pretrain_part_"+str(SPLIT_INDEX) + ".json"

with open(new_file_name, 'w') as file:
    json.dump(used_vids, file, indent=4)